Import Libraries

In [8]:
import turicreate as tc
import pandas as pd
import ast

<b>Set GPU for Recommendation system</b>

In [2]:
tc.config.set_num_gpus(1)

<b>Load DataSet</b>

In [3]:
user_data     = tc.SFrame.read_csv('movielens/ratings.csv')
item_data     = tc.SFrame.read_csv('movielens/movies_metadata.csv')
#credits_data  = tc.SFrame.read_csv('movielens/credits.csv')
keywords_data = tc.SFrame.read_csv('movielens/keywords.csv')

Finished parsing file /Users/sachin/Documents/sachin/Education/SMU/Term4/7331/DataMining/ML7331/movielens/ratings.csv

Parsing completed. Parsed 100 lines in 0.522223 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2006281 lines. Lines per second: 1.99517e+06

Read 21225249 lines. Lines per second: 3.31984e+06

...

Finished parsing file /Users/sachin/Documents/sachin/Education/SMU/Term4/7331/DataMining/ML7331/movielens/keywords.csv

Parsing completed. Parsed 100 lines in 0.129355 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,list]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/sachin/Documents/sachin/Education/SMU/Term4/7331/DataMining/ML7331/movielens/keywords.csv

Parsing completed. Parsed 46419 lines in 0.207573 secs.

In [11]:
credits = pd.read_csv('movielens/credits.csv')

# Drop rows where both cast and crew are empty
credits = credits[~((credits['cast'].str.contains('\[\]')) & (credits['crew'].str.contains('\[\]')))]

def dedupe(df):
    dupes = len(df[df.duplicated(subset=['id'])])
    print('Removed', dupes, 'duplicate rows.\n')
    return df[~df.duplicated(subset=['id'])]

# Remove duplicates:
credits = dedupe(credits)

def get_credits(crew_json):
    crew = ast.literal_eval(crew_json.crew)
    cast = ast.literal_eval(crew_json.cast)
    directors = []
    producers = []
    actors = []
    for member in crew:
#        if member is np.nan:
#            member = '[]'
        if 'Director' in member['job'] and member['job'] is not np.nan:
            directors.append(member['name'])
        if 'Producer' in member['job'] and member['job'] is not np.nan:
            producers.append(member['name'])
    for actor in cast:
#        if actor is np.nan:
#            actor = '[]'
        if actor['name'] is not np.nan:
            actors.append(actor['name'])

    return  pd.Series([directors, producers, actors])

credits[['directors','producers','actors']] = credits.apply(get_credits, axis=1)

def make_dict(lst):
    if lst is np.nan:
        return np.nan
    else:
        return { i : 'name' for i in lst }
        #return {i:lst[i] for i in range(0, len(lst))}

credits['actors_dict'] = credits.actors.apply(make_dict)
credits['producers_dict'] = credits.producers.apply(make_dict)
credits['directors_dict'] = credits.directors.apply(make_dict)

credits.head()

Removed 44 duplicate rows.



,cast,crew,id,directors,producers,actors,actors_dict,producers_dict,directors_dict
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[John Lasseter, Ash Brannon, William Reeves, R...","[Bonnie Arnold, Ed Catmull, Ralph Guggenheim, ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","{'Tom Hanks': 'name', 'Tim Allen': 'name', 'Do...","{'Bonnie Arnold': 'name', 'Ed Catmull': 'name'...","{'John Lasseter': 'name', 'Ash Brannon': 'name..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Joe Johnston, Thomas E. Ackerman]","[Larry J. Franco, Scott Kroopf, Ted Field, Rob...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","{'Robin Williams': 'name', 'Jonathan Hyde': 'n...","{'Larry J. Franco': 'name', 'Scott Kroopf': 'n...","{'Joe Johnston': 'name', 'Thomas E. Ackerman':..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch],[],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","{'Walter Matthau': 'name', 'Jack Lemmon': 'nam...",{},{'Howard Deutch': 'name'}
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,[Forest Whitaker],"[Ronald Bass, Ezra Swerdlow, Deborah Schindler...","[Whitney Houston, Angela Bassett, Loretta Devi...","{'Whitney Houston': 'name', 'Angela Bassett': ...","{'Ronald Bass': 'name', 'Ezra Swerdlow': 'name...",{'Forest Whitaker': 'name'}
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[Elliot Davis, Charles Shyer]",[Nancy Meyers],"[Steve Martin, Diane Keaton, Martin Short, Kim...","{'Steve Martin': 'name', 'Diane Keaton': 'name...",{'Nancy Meyers': 'name'},"{'Elliot Davis': 'name', 'Charles Shyer': 'name'}"


In [ ]:
keywords_data = tc.SFrame(credits_data)

## Convert Dictionaries to list

### Belongs to collection

In [12]:
item_data['collection_name'] = [ it['belongs_to_collection']['name'] if it['belongs_to_collection'] != None else " " for it in item_data.select_columns(['belongs_to_collection'])] 

In [13]:
item_data.head()

adult,belongs_to_collection,budget,genres,homepage,id
FALSE,"{'id': 10194, 'name':'Toy Story Collection', ...",30000000,"[{'id': 16, 'name':'Animation'}, {'id': 35, ...",http://toystory.disney.com/toy-story ...,862
FALSE,None,65000000,"[{'id': 12, 'name':'Adventure'}, {'id': 14, ...",,8844
FALSE,"{'id': 119050, 'name':'Grumpy Old Men ...",0,"[{'id': 10749, 'name':'Romance'}, {'id': 35, ...",,15602
FALSE,None,16000000,"[{'id': 35, 'name':'Comedy'}, {'id': 18, ...",,31357
FALSE,"{'id': 96871, 'name':'Father of the Bride ...",0,"[{'id': 35, 'name':'Comedy'}] ...",,11862
FALSE,None,60000000,"[{'id': 28, 'name':'Action'}, {'id': 80, ...",,949
FALSE,None,58000000,"[{'id': 35, 'name':'Comedy'}, {'id': 10749, ...",,11860
FALSE,None,0,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,45325
FALSE,None,35000000,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,9091
FALSE,"{'id': 645, 'name':'James Bond Collection', ...",58000000,"[{'id': 12, 'name':'Adventure'}, {'id': 28, ...",http://www.mgm.com/view/movie/757/Goldeneye/ ...,710


### Genres

In [16]:
gen_l = []

for itms in item_data.select_columns(['genres']):
    #items[row]['genres_l']  = items[row]['genres_l'].apply([ it['name'] for it in itms['genres']])
    if(itms['genres'] != None):
        gen_l.append([ it['name'] if it['name'] !=None else " " for it in itms['genres']])
    else:
        gen_l.append([" "])


item_data['genres_l'] = gen_l

item_data


dtype: list
Rows: 45462
[['Animation', 'Comedy', 'Family'], ['Adventure', 'Fantasy', 'Family'], ['Romance', 'Comedy'], ['Comedy', 'Drama', 'Romance'], ['Comedy'], ['Action', 'Crime', 'Drama', 'Thriller'], ['Comedy', 'Romance'], ['Action', 'Adventure', 'Drama', 'Family'], ['Action', 'Adventure', 'Thriller'], ['Adventure', 'Action', 'Thriller'], ['Comedy', 'Drama', 'Romance'], ['Comedy', 'Horror'], ['Family', 'Animation', 'Adventure'], ['History', 'Drama'], ['Action', 'Adventure'], ['Drama', 'Crime'], ['Drama', 'Romance'], ['Crime', 'Comedy'], ['Crime', 'Comedy', 'Adventure'], ['Action', 'Comedy', 'Crime'], ['Comedy', 'Thriller', 'Crime'], ['Drama', 'Thriller'], ['Action', 'Adventure', 'Crime', 'Thriller'], ['Drama', 'Fantasy', 'Science Fiction', 'Thriller'], ['Drama', 'Romance'], ['Drama'], ['Comedy', 'Drama', 'Family'], ['Drama', 'Romance'], ['Fantasy', 'Science Fiction', 'Adventure'], ['Drama', 'Crime'], ['Drama', 'Crime'], ['Science Fiction', 'Thriller', 'Mystery'], ['Romance', 'Adve

In [17]:
item_data.head()

adult,belongs_to_collection,budget,genres,homepage,id
FALSE,"{'id': 10194, 'name':'Toy Story Collection', ...",30000000,"[{'id': 16, 'name':'Animation'}, {'id': 35, ...",http://toystory.disney.com/toy-story ...,862
FALSE,None,65000000,"[{'id': 12, 'name':'Adventure'}, {'id': 14, ...",,8844
FALSE,"{'id': 119050, 'name':'Grumpy Old Men ...",0,"[{'id': 10749, 'name':'Romance'}, {'id': 35, ...",,15602
FALSE,None,16000000,"[{'id': 35, 'name':'Comedy'}, {'id': 18, ...",,31357
FALSE,"{'id': 96871, 'name':'Father of the Bride ...",0,"[{'id': 35, 'name':'Comedy'}] ...",,11862
FALSE,None,60000000,"[{'id': 28, 'name':'Action'}, {'id': 80, ...",,949
FALSE,None,58000000,"[{'id': 35, 'name':'Comedy'}, {'id': 10749, ...",,11860
FALSE,None,0,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,45325
FALSE,None,35000000,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,9091
FALSE,"{'id': 645, 'name':'James Bond Collection', ...",58000000,"[{'id': 12, 'name':'Adventure'}, {'id': 28, ...",http://www.mgm.com/view/movie/757/Goldeneye/ ...,710


### Spoken Langages

In [74]:
sp_l = []

for itms in item_data.select_columns(['spoken_languages']):
    #items[row]['genres_l']  = items[row]['genres_l'].apply([ it['name'] for it in itms['genres']])
    if(itms['spoken_languages'] != None):
        sp_l.append([ it['name'] if it['name'] !=None else " " for it in itms['spoken_languages']])
    else:
        sp_l.append([" "])


item_data['spoken_l'] = sp_l

item_data

adult,belongs_to_collection,budget,genres,homepage,id
FALSE,"{'id': 10194, 'name':'Toy Story Collection', ...",30000000,"[{'id': 16, 'name':'Animation'}, {'id': 35, ...",http://toystory.disney.com/toy-story ...,862
FALSE,None,65000000,"[{'id': 12, 'name':'Adventure'}, {'id': 14, ...",,8844
FALSE,"{'id': 119050, 'name':'Grumpy Old Men ...",0,"[{'id': 10749, 'name':'Romance'}, {'id': 35, ...",,15602
FALSE,None,16000000,"[{'id': 35, 'name':'Comedy'}, {'id': 18, ...",,31357
FALSE,"{'id': 96871, 'name':'Father of the Bride ...",0,"[{'id': 35, 'name':'Comedy'}] ...",,11862
FALSE,None,60000000,"[{'id': 28, 'name':'Action'}, {'id': 80, ...",,949
FALSE,None,58000000,"[{'id': 35, 'name':'Comedy'}, {'id': 10749, ...",,11860
FALSE,None,0,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,45325
FALSE,None,35000000,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,9091
FALSE,"{'id': 645, 'name':'James Bond Collection', ...",58000000,"[{'id': 12, 'name':'Adventure'}, {'id': 28, ...",http://www.mgm.com/view/movie/757/Goldeneye/ ...,710


In [77]:
pc_l = []

for itms in item_data.select_columns(['production_companies']):
    #items[row]['genres_l']  = items[row]['genres_l'].apply([ it['name'] for it in itms['genres']])
    if(itms['production_companies'] != None):
        pc_l.append([ it['name'] if it['name'] !=None else " " for it in itms['production_companies']])
    else:
        pc_l.append([" "])
        
item_data['prod_companies'] = pc_l


In [80]:
item_data.head()

adult,belongs_to_collection,budget,genres,homepage,id
FALSE,"{'id': 10194, 'name':'Toy Story Collection', ...",30000000,"[{'id': 16, 'name':'Animation'}, {'id': 35, ...",http://toystory.disney.com/toy-story ...,862
FALSE,None,65000000,"[{'id': 12, 'name':'Adventure'}, {'id': 14, ...",,8844
FALSE,"{'id': 119050, 'name':'Grumpy Old Men ...",0,"[{'id': 10749, 'name':'Romance'}, {'id': 35, ...",,15602
FALSE,None,16000000,"[{'id': 35, 'name':'Comedy'}, {'id': 18, ...",,31357
FALSE,"{'id': 96871, 'name':'Father of the Bride ...",0,"[{'id': 35, 'name':'Comedy'}] ...",,11862
FALSE,None,60000000,"[{'id': 28, 'name':'Action'}, {'id': 80, ...",,949
FALSE,None,58000000,"[{'id': 35, 'name':'Comedy'}, {'id': 10749, ...",,11860
FALSE,None,0,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,45325
FALSE,None,35000000,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,9091
FALSE,"{'id': 645, 'name':'James Bond Collection', ...",58000000,"[{'id': 12, 'name':'Adventure'}, {'id': 28, ...",http://www.mgm.com/view/movie/757/Goldeneye/ ...,710


In [81]:
pc_cl = []

for itms in item_data.select_columns(['production_countries']):
    if(itms['production_countries'] != None):
        pc_cl.append([ it['name'] if it['name'] !=None else " " for it in itms['production_countries']])
    else:
        pc_cl.append([" "])
pc_cl        

[['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['Germany', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United Kingdom', 'United States of America'],
 ['United States of America'],
 ['France', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['France', 'Germany', 'Italy', 'United States of America'],
 ['France', 'United States of America'],
 ['United Kingdom', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['France', 'United States of America'],
 ['United States of America'],
 ['United States of America'],
 ['Italy'],
 ['United States of America'],
 ['United Kingdom', 'France'],
 ['France', 'Germany', 'Spain'],
 ['China', 

In [82]:
item_data['prod_countries'] = pc_cl

item_data.head()

adult,belongs_to_collection,budget,genres,homepage,id
FALSE,"{'id': 10194, 'name':'Toy Story Collection', ...",30000000,"[{'id': 16, 'name':'Animation'}, {'id': 35, ...",http://toystory.disney.com/toy-story ...,862
FALSE,None,65000000,"[{'id': 12, 'name':'Adventure'}, {'id': 14, ...",,8844
FALSE,"{'id': 119050, 'name':'Grumpy Old Men ...",0,"[{'id': 10749, 'name':'Romance'}, {'id': 35, ...",,15602
FALSE,None,16000000,"[{'id': 35, 'name':'Comedy'}, {'id': 18, ...",,31357
FALSE,"{'id': 96871, 'name':'Father of the Bride ...",0,"[{'id': 35, 'name':'Comedy'}] ...",,11862
FALSE,None,60000000,"[{'id': 28, 'name':'Action'}, {'id': 80, ...",,949
FALSE,None,58000000,"[{'id': 35, 'name':'Comedy'}, {'id': 10749, ...",,11860
FALSE,None,0,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,45325
FALSE,None,35000000,"[{'id': 28, 'name':'Action'}, {'id': 12, ...",,9091
FALSE,"{'id': 645, 'name':'James Bond Collection', ...",58000000,"[{'id': 12, 'name':'Adventure'}, {'id': 28, ...",http://www.mgm.com/view/movie/757/Goldeneye/ ...,710


## Keywords data 

In [84]:


key_l = []

for itms in keywords_data.select_columns(['keywords']):
    if(itms['keywords'] != None):
        key_l.append([ it['name'] if it['name'] !=None else " " for it in itms['keywords']])
    else:
        key_l.append([" "])
key_l        

[['jealousy',
  'toy',
  'boy',
  'friendship',
  'friends',
  'rivalry',
  'boy next door',
  'new toy',
  'toy comes to life'],
 ['board game',
  'disappearance',
  "based on children's book",
  'new home',
  'recluse',
  'giant insect'],
 ['fishing', 'best friend', 'duringcreditsstinger', 'old men'],
 ['based on novel',
  'interracial relationship',
  'single mother',
  'divorce',
  'chick flick'],
 ['baby',
  'midlife crisis',
  'confidence',
  'aging',
  'daughter',
  'mother daughter relationship',
  'pregnancy',
  'contraception',
  'gynecologist'],
 ['robbery',
  'detective',
  'bank',
  'obsession',
  'chase',
  'shooting',
  'thief',
  'honor',
  'murder',
  'suspense',
  'heist',
  'betrayal',
  'money',
  'gang',
  'cat and mouse',
  'criminal mastermind',
  'cult film',
  'ex-con',
  'heist movie',
  'one last job',
  'loner',
  'bank job',
  'neo-noir',
  'gun fight',
  'crime epic'],
 ['paris',
  'brother brother relationship',
  'chauffeur',
  'long island',
  'fusion',

In [87]:
keywords_data['keywords_l'] = key_l

In [88]:
keywords_data

id,keywords,keywords_l
862,"[{'id': 931, 'name':'jealousy'}, {'id': 4 ...","[jealousy, toy, boy,friendship, friends, ..."
8844,"[{'id': 10090, 'name':'board game'}, {'id': ...","[board game,disappearance, based on ..."
15602,"[{'id': 1495, 'name':'fishing'}, {'id': 12 ...","[fishing, best friend,duringcreditsstinger, ..."
31357,"[{'id': 818, 'name':'based on novel'}, {' ...","[based on novel,interracial relations ..."
11862,"[{'id': 1009, 'name':'baby'}, {'id': 1599, ...","[baby, midlife crisis,confidence, aging, ..."
949,"[{'id': 642, 'name':'robbery'}, {'id': 703, ...","[robbery, detective,bank, obsession, chase, ..."
11860,"[{'id': 90, 'name':'paris'}, {'id': 380, ...","[paris, brother brotherrelationship, chauffeur, ..."
45325,[],[]
9091,"[{'id': 949, 'name':'terrorist'}, {'id': ...","[terrorist, hostage,explosive, vice ..."
710,"[{'id': 701, 'name':'cuba'}, {'id': 769, ...","[cuba, falsely accused,secret identity, comp ..."


## Credits data

In [89]:
credits

,cast,crew,id,directors,producers,actors,actors_dict,producers_dict,directors_dict
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[John Lasseter, Ash Brannon, William Reeves, R...","[Bonnie Arnold, Ed Catmull, Ralph Guggenheim, ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","{'Tom Hanks': 'name', 'Tim Allen': 'name', 'Do...","{'Bonnie Arnold': 'name', 'Ed Catmull': 'name'...","{'John Lasseter': 'name', 'Ash Brannon': 'name..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Joe Johnston, Thomas E. Ackerman]","[Larry J. Franco, Scott Kroopf, Ted Field, Rob...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","{'Robin Williams': 'name', 'Jonathan Hyde': 'n...","{'Larry J. Franco': 'name', 'Scott Kroopf': 'n...","{'Joe Johnston': 'name', 'Thomas E. Ackerman':..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch],[],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","{'Walter Matthau': 'name', 'Jack Lemmon': 'nam...",{},{'Howard Deutch': 'name'}
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,[Forest Whitaker],"[Ronald Bass, Ezra Swerdlow, Deborah Schindler...","[Whitney Houston, Angela Bassett, Loretta Devi...","{'Whitney Houston': 'name', 'Angela Bassett': ...","{'Ronald Bass': 'name', 'Ezra Swerdlow': 'name...",{'Forest Whitaker': 'name'}
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[Elliot Davis, Charles Shyer]",[Nancy Meyers],"[Steve Martin, Diane Keaton, Martin Short, Kim...","{'Steve Martin': 'name', 'Diane Keaton': 'name...",{'Nancy Meyers': 'name'},"{'Elliot Davis': 'name', 'Charles Shyer': 'name'}"
...,...,...,...,...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,"[Hamid Nematollah, Babak Ardalan]",[Hamid Nematollah],"[Leila Hatami, Kourosh Tahami, Elham Korda]","{'Leila Hatami': 'name', 'Kourosh Tahami': 'na...",{'Hamid Nematollah': 'name'},"{'Hamid Nematollah': 'name', 'Babak Ardalan': ..."
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,[Lav Diaz],[],"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...","{'Angel Aquino': 'name', 'Perry Dizon': 'name'...",{},{'Lav Diaz': 'name'}
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,"[Mark L. Lester, João Fernandes]",[],"[Erika Eleniak, Adam Baldwin, Julie du Page, J...","{'Erika Eleniak': 'name', 'Adam Baldwin': 'nam...",{},"{'Mark L. Lester': 'name', 'João Fernandes': '..."
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,[Yakov Protazanov],[Joseph N. Ermolieff],"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...","{'Iwan Mosschuchin': 'name', 'Nathalie Lissenk...",{'Joseph N. Ermolieff': 'name'},{'Yakov Protazanov': 'name'}
